In [1]:
import pandas as pd
import nltk
import spacy
import gensim
import seaborn as sb
from gensim import corpora, models, similarities
from spacy.lang.en import English
from nltk.corpus import wordnet as wn

In [2]:
data=pd.read_csv("DurhamCountyOriginalDataFY20.csv",)

In [3]:
data.head()

,Unnamed: 0,page_number,word
0,1,1,approved
1,2,1,budget
2,3,1,fy
3,4,2,durham
4,5,2,county


In [4]:
data.drop(data.columns[0], axis=1)

,page_number,word
0,1,approved
1,1,budget
2,1,fy
3,2,durham
4,2,county
5,2,north
6,2,carolina
7,2,fy
8,2,approved
9,2,budget


In [5]:
data.describe()

,Unnamed: 0,page_number
count,97576.00000,97576.000000
mean,48788.50000,161.767474
std,28167.90927,94.198533
min,1.00000,1.000000
25%,24394.75000,78.000000
50%,48788.50000,158.000000
75%,73182.25000,243.000000
max,97576.00000,323.000000


In [6]:
#cleaning 
spacy.load('en')
parser=English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.orth_.startswith('http'):
            lda_tokens.append('com')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [7]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\messi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
def get_lemma(word):
    lemma=wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

In [9]:
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [10]:
#filtering stopwords
nltk.download('stopwords')
stop=set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\messi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
#preparing for lda
def prepare_lda(text):
    tokens=tokenize(text)
    tokens=[token for token in tokens if len(token)>4]
    tokens=[token for token in tokens if token not in stop]
    tokens=[get_lemma(token) for token in tokens]
    return tokens

In [12]:
#creating tokens
import random
text_data=[]
with open('DurhamCountyOriginalDataFY20.csv') as f:
    for line in f:
        tokens=prepare_lda(line)
        if random.random()>.99:
            print(tokens)
            text_data.append(tokens)

['66",2,"jay']
['224",3,"register']
['373",4,"officials']
['569",4,"is']
['577",4,"was']
['607",4,"high']
['623",4,"county']
['628",4,"of']
['705",4,"as']
['833",4,"the']
['894",4,"resources']
['914",4,"for']
['1158",6,"united']
['1221",7,"shows']
['1242",7,"funds']
['1252",7,"is']
['1272",7,"include']
['1294",7,"of']
['1307",7,"a']
['1328",7,"show']
['1459",7,"an']
['1593",7,"adequate']
['1711",7,"employees']
['1867",8,"the']
['2046",8,"capital']
['2392",9,"fee']
['2567",10,"mental']
['2705",11,"fund']
['2740",12,"debt']
['2769",12,"contacts']
['2838",13,"a']
['2846",13,"a']
['2875",13,"our']
['2911",14,"county']
['3166",14,"in']
['3206",14,"developing']
['3310",14,"in']
['3361",14,"matter']
['3399",15,"tax']
['3427",15,"cents']
['3456",15,"is']
['3485",15,"neutral']
['3533",15,"new']
['3608",15,"in']
['3649",15,"challenge']
['3678",15,"consumed']
['3782",15,"county']
['3887",16,"in']
['3892",16,"state']
['3894",16,"this']
['4153",16,"our']
['4187",16,"october']
['4198",16,"of']
['431

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 5865: character maps to <undefined>

In [13]:
#creating dictionary
from gensim import corpora
dictionary=corpora.Dictionary(text_data)
corpus= [dictionary.doc2bow(text) for text in text_data]
import pickle 
pickle.dump(corpus, open('corpus.pkl', 'wb'))

In [14]:
#example
ldamodel=gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=6)
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} word: {}'.format(idx, topic))

Topic: 0 word: 0.052*"9655",27,"with" + 0.052*"7963",24,"alliance" + 0.052*"5807",21,"durham" + 0.052*"833",4,"the" + 0.052*"2392",9,"fee" + 0.052*"1242",7,"funds" + 0.052*"6848",22,"with" + 0.052*"6650",22,"children" + 0.052*"7212",23,"better" + 0.005*"4187",16,"october"
Topic: 1 word: 0.050*"3782",15,"county" + 0.050*"10603",29,"by" + 0.050*"2567",10,"mental" + 0.050*"9817",27,"operational" + 0.050*"4823",18,"tax" + 0.050*"5540",20,"goal" + 0.050*"2838",13,"a" + 0.050*"5560",20,"support" + 0.050*"569",4,"is" + 0.050*"1593",7,"adequate"
Topic: 2 word: 0.040*"7794",24,"manager" + 0.040*"5031",19,"of" + 0.040*"1221",7,"shows" + 0.040*"2911",14,"county" + 0.040*"10546",29,"and" + 0.040*"5368",20,"plan" + 0.040*"2046",8,"capital" + 0.040*"8070",25,"center" + 0.040*"705",4,"as" + 0.040*"6494",22,"a"
Topic: 3 word: 0.040*"10998",30,"is" + 0.040*"5455",20,"earlier" + 0.040*"224",3,"register" + 0.040*"1459",7,"an" + 0.040*"9787",27,"government" + 0.040*"5205",19,"of" + 0.040*"4972",19,"of" + 

In [16]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)